In [10]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from datetime import date

def getGeneralDataframe(jsonPath, csvPath):

  """ Funcion to obtain a dataFrame with the data of the Json file
      Parameters: json path """

  csvdata = pd.read_csv(csvPath)
  csvdata = csvdata.set_index("username")

  with open(jsonPath) as f:
    data = json.load(f)

  #Create general data frame
  df = pd.DataFrame(columns=[ "gender","user", "followers","likes", "score","item0", "item1","date" ])

  #Put the data in the data frame
  for i in range(0,len(data['output'])):
    df.loc[i] = [data['output'][i]["gender"],data['output'][i]["user"], data['output'][i]["followers"],data['output'][i]["likes"], csvdata.loc[data["output"][i]["user"]].score, data['output'][i]["item0"], data['output'][i]["item1"],data['output'][i]["date"]]

  #Eliminate rows which no value in likes from json
  df['likes'].replace('', np.nan, inplace=True)
  #df['item0'].replace('N/A N/A', np.nan, inplace=True)
  #print(df)
  df.dropna(subset=['likes'], inplace=True)
  #Se resetean los indices para el siguiente for
  df.reset_index(inplace=True, drop=True)
  
  #Change the type of followers and likes variables (String to int)
  df['followers']=df['followers'].astype(int)
  df['likes']=df['likes'].astype(int)
  df['date']=pd.to_datetime(df['date'])
  
  return df






In [11]:
def getFilteredDataframe(df,gender):
  """ Funcion to obtain a dataFrame filtering the general dataframe by a specific gender
      Parameters: Dataframe, gender (writen in the same format that the json gender parameter) """
  df1 = df[df["gender"] == gender]
  df1.reset_index(inplace=True, drop=True)
  
  return df1

In [12]:
def getPossibleTrend(df):
      
  #Create new colum in panda series format in order to calculate difference in days
  #today = pd.to_datetime("today")
  #df['diff_days']= (today - df['date']).dt.days + 1
  

  # TODO: Define better formula
  # TODO: Change 36 to 9 and define better day mapping
  df['post_engagement'] = (df['likes'] / df['followers'])*100
  #* (37 - df['diff_days'])

  m0, m1 ,m2, m3, m4, m5 = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
  total_avg0, total_avg1, total_avg2, total_avg3, total_avg4, total_avg5 = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
  
  df['avg_interval']=0.0


  for i in range(0, len(df['followers'])):

    if(0 <= df['followers'][i] < 5000):
      total_avg0 = df['post_engagement'][i] + total_avg0
      m0 = m0 + 1
    
    elif (5000 <= df['followers'][i] < 20000):
      total_avg1 = df['post_engagement'][i] + total_avg1
      m1 = m1 + 1

    elif (20000 <= df['followers'][i] < 50000):
      total_avg2 = df['post_engagement'][i] + total_avg2
      m2 = m2 + 1

    elif (50000 <= df['followers'][i] < 200000):
      total_avg3 = df['post_engagement'][i] + total_avg3
      m3 = m3 + 1  

    elif (200000 <= df['followers'][i] < 500000):
      total_avg4 = df['post_engagement'][i] + total_avg4
      m4 = m4 + 1 

    elif (df['followers'][i] > 500000):
      total_avg5 = df['post_engagement'][i] + total_avg5
      m5 = m5 + 1

  '''print(m0,m1,m2,m3,m4,m5)
  print(total_avg0,total_avg1,total_avg2,total_avg3,total_avg4,total_avg5)
  print(total_avg0/m0,total_avg1/m1,total_avg2/m2,total_avg3/m3,total_avg4/m4,total_avg5/m5)'''


  #average_engagement per row depending on followers (calculated before avg_eng), we only introduce those values per intervals
  '''OJO CON LO SIGUIENTE (!!AREGLAR!!) --> Si m0,1,2,3=0, error ---> Posible solucion hacer fuera if(m0>1): x=0 else: x=total_avg0/m0 e igual para los otros cn otra letra
    otra posible solución es --> cambiar en el siguiente for el clave0 y hacer la division directamente, es mas costoso? '''
  #intervals_average = {"clave0":total_avg0/m0, "clave1":total_avg1/m1, "clave2":total_avg2/m2, "clave3":total_avg3/m3, "clave4":total_avg4/m4,"clave5":total_avg5/m5}
  df['value']=0.0
  j=0

  for i in range(0, len(df['followers'])):
    if((0 <= df['followers'][i] < 5000) and (m0 > 0)):
      df.loc[j,'avg_interval']= total_avg0/m0
      '''intervals_average['clave0']'''
      df.loc[j,'value']=(df['post_engagement'][i] - df['avg_interval'][i])*0.5
      j=j+1

    elif((5000 <= df['followers'][i] < 20000) and (m1 > 0)):
      df.loc[j,'avg_interval'] = total_avg1/m1
      df.loc[j,'value']=(df['post_engagement'][i] - df['avg_interval'][i])*0.5
      j=j+1

    elif((20000 <= df['followers'][i] < 50000) and (m2 > 0)):
      df.loc[j,'avg_interval'] = total_avg2/m2
      df.loc[j,'value']=(df['post_engagement'][i] - df['avg_interval'][i])*0.5
      j=j+1

    elif((50000 <= df['followers'][i] < 200000) and (m3 > 0)):
      df.loc[j,'avg_interval'] = total_avg3/m3
      df.loc[j,'value']=(df['post_engagement'][i] - df['avg_interval'][i])*0.5
      j=j+1

    elif((200000 <= df['followers'][i] < 500000) and (m4 > 0)):
      df.loc[j,'avg_interval'] = total_avg4/m4
      df.loc[j,'value']=(df['post_engagement'][i] - df['avg_interval'][i])*0.5
      j=j+1

    elif ((df['followers'][i] > 500000) and (m5 > 0)):
      df.loc[j,'avg_interval'] = total_avg5/m5
      df.loc[j,'value']=(df['post_engagement'][i] - df['avg_interval'][i])*0.5
      j=j+1

  
  #Esto era una prueba con unos valores, falta acabar los intervalos, se puede meter en el bucle anterior (ya metido jose)

  '''for i in range(0, len(df['followers'])):
      if((df['followers'][i] >= 0) and (df['followers'][i] < 5000)):
        #lo comparamos con la media del intervalo y ponderamos según nº followers, +followers +peso
        #TODO define intervals weight (0.1 de prueba)
        df['value'][i]=(df['post_engagement'][i] - df['avg_interval'][i])*0.1     

      elif ((df['followers'][i] >= 5000) and (df['followers'][i] < 20000000)):
          df['value'][i]=(df['post_engagement'][i] - df['avg_interval'][i])*0.5'''
  return df

In [13]:
def getLabelsDF(df,item):
  """ Parameters: Dataframe, name ofº the column that contains the item. Example 'item0' """
  df_item = pd.DataFrame(columns=["item","counts"])
  j=0
  for i in range(0, len(df[item].value_counts())):
    df_item.loc[j]=df[item].value_counts().index.tolist()[i],df[item].value_counts()[i]
    j=j+1

  return df_item

In [30]:
if __name__ == "__main__":
  postsDF = getGeneralDataframe("recognition-output.json", "influencers.csv")
  df_w= getFilteredDataframe(postsDF,"WOMAN").copy()
  df_m= getFilteredDataframe(postsDF,"MAN").copy()

  possible_w = getPossibleTrend(df_w)
  possible_m = getPossibleTrend(df_m)
  
  topLabelsDF = getLabelsDF(possible_w,'item0')
  bottomLabelsDF = getLabelsDF(possible_w,'item1')

In [32]:

#postsDF
#topLabelsDF
#bottomLabelsDF
#df_w
#df_m
#possible_w
#pd.set_option('display.max_rows', None)
possible_m



,gender,user,followers,likes,score,item0,item1,date,post_engagement,avg_interval,value
0,MAN,artur__fit,901124,14714,0.5,jersey negro,pantalones gris,2021-07-11,1.632850,0.472713,0.580068
1,MAN,artur__fit,901124,14743,0.5,jersey azul claro,pantalones lila,2021-05-11,1.636068,0.472713,0.581677
2,MAN,artur__fit,901124,19701,0.5,abrigo_manga_larga negro,pantalones khaki,2021-02-11,2.186270,0.472713,0.856778
3,MAN,artur__fit,901124,19660,0.5,jersey negro,pantalones negro,2021-10-27,2.181720,0.472713,0.854503
4,MAN,magno_scavo,2557486,32418,0.5,jersey gris,pantalones negro,2021-07-11,1.267573,0.472713,0.397430
5,MAN,magno_scavo,2557486,59096,0.5,jersey beige,pantalones negro,2021-01-11,2.310707,0.472713,0.918997
6,MAN,lucadebrito,73920,1193,0.5,jersey negro,pantalones negro,2021-04-11,1.613907,2.317272,-0.351683
7,MAN,lucadebrito,73920,1545,0.5,abrigo_manga_larga negro,pantalones verde oscuro,2021-02-11,2.090097,2.317272,-0.113587
8,MAN,lucadebrito,73920,1383,0.5,camiseta_manga_corta negro,pantalones negro,2021-10-31,1.870942,2.317272,-0.223165
9,MAN,lucadebrito,73920,2543,0.5,jersey blanco,pantalones gris,2021-10-30,3.440206,2.317272,0.561467
